In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [11]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
pinfan       12
boardname    esp8266sens2
connection1  DoESLiverpool,decafbad00,10.0.100.1

Sent 7 lines (173 bytes) to config.txt.


In [7]:
%sendtofile --source ../Sensor_Kennel/BME280_funcs.py

Sent 79 lines (3304 bytes) to BME280_funcs.py.


In [5]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 76 lines (2732 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [26]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [27]:
%sendtofile main.py

import time, itertools
from machine import Pin
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from stdmqttas import pinled, flashpinled, shortmac, topicstem, topicstatus
from mqtt_as import MQTTClient
from BME280_funcs import bme280init, readBME280

import network
network.WLAN().active(0)  # disable the connection at startup
pinfan = Pin(int(fconfig["pinfan"]), Pin.OUT)
flashpinled(5, 300, 300)

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
print(list(map(hex, i2c.scan())))

topicfan = topicstem+"/fan"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicbme280A = topicstem+'/bme280/A'
topicbme280B = topicstem+'/bme280/B'


# dereferenced so we can swap the sample rate so we can 
# see how it affects the temperature and fan combinations
bmeparams = [[0x76, topicbme280A, 500],
             [0x77, topicbme280B, 60000]]

async def fantask():
    pinfan.value(0)
    while not client.isconnected():
        pinfan.value(1-pinfan.value())
        await asyncio.sleep_ms(1000)
    pinfan.value(0)
    for i in itertools.count():
        print("fan", i)
        if (i%3) == 2:
            bmeparams[0][2], bmeparams[1][2] = bmeparams[1][2], bmeparams[0][2]
            print("fliptimes", bmeparams[0][2], bmeparams[1][2])
        if (i%4) == 1:
            v = 1-pinfan.value()
            print(topicfan, v)
            await client.publish(topicfan, str(v), retain=True)
            pinfan.value(v)
        await asyncio.sleep_ms(15*60*1000)
    
async def bme280task(k):
    btoinit = True
    addr, topic = bmeparams[k][:2]
    for i in itertools.count():
        try:
            if btoinit:
                bme280init(i2c, addr)
                btoinit = False
                await asyncio.sleep_ms(100)
            meas = readBME280(addr)
            payload = " ".join(map(str, meas))
            if client.isconnected():
                print(topic, meas)
                await client.publish(topic, payload)
        except OSError as e:
            print("bme280task", e)
            btoinit = True
        if bmeparams[k][2] < 1000:
            pinled.value((i%10) < (1 if client.isconnected() else 5))
        await asyncio.sleep_ms(bmeparams[k][2])

        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
    
aloop = asyncio.get_event_loop()
aloop.create_task(bme280task(0))
aloop.create_task(bme280task(1))
aloop.create_task(fantask())
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()



Sent 93 lines (3043 bytes) to main.py.
